In [1]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [3]:
from model import ThreeLayerConvNet
from trainer import train_loop
from data import import_data, Dataset

In [4]:
X_train, y_train, X_test, y_test, X_val, y_val = import_data()
train_dset = Dataset(X_train, y_train, 16, shuffle = True )
val_dset = Dataset(X_test, y_test, 16, shuffle = False) ##NOTE: Validating on test for some reason
test_dset = Dataset(X_val, y_val, 15, shuffle = False)


In [21]:
for t, (x_np, y_np) in enumerate(train_dset):
    print (y_np)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0.]


In [22]:
print (y_val)

[0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]


In [38]:
def model_init_fn():
    channel_1, channel_2, num_classes = 64,64,2
    
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Conv2D(channel_1, 5, strides = 1,
                                padding = "same", activation = "relu", kernel_initializer = initializer),
        tf.keras.layers.Conv2D(channel_2, 3, strides = 1,
                               padding = "same", activation = "relu", kernel_initializer = initializer),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_classes, activation = "softmax", kernel_initializer = initializer )
        
    ]
    model = tf.keras.Sequential(layers)
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = tf.keras.optimizers.SGD(learning_rate = learning_rate, momentum = 0.9, nesterov = True)
    return optimizer


In [39]:
model = model_init_fn()
model.compile(optimizer_init_fn(),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=16, epochs=10, validation_data=(X_test, y_test))


Train on 116 samples, validate on 69 samples
Epoch 1/10
116/116 [==============================] - 13s 109ms/sample - loss: 1505.6705 - sparse_categorical_accuracy: 0.5690 - val_loss: 317.3890 - val_sparse_categorical_accuracy: 0.9130
Epoch 2/10
116/116 [==============================] - 7s 62ms/sample - loss: 3361946.4192 - sparse_categorical_accuracy: 0.5431 - val_loss: 0.3150 - val_sparse_categorical_accuracy: 0.9130
Epoch 3/10
116/116 [==============================] - 7s 62ms/sample - loss: 0.4395 - sparse_categorical_accuracy: 0.8793 - val_loss: 0.5559 - val_sparse_categorical_accuracy: 0.9130
Epoch 4/10
116/116 [==============================] - 7s 62ms/sample - loss: 0.4482 - sparse_categorical_accuracy: 0.8707 - val_loss: 0.4001 - val_sparse_categorical_accuracy: 0.9130
Epoch 5/10
116/116 [==============================] - 7s 62ms/sample - loss: 0.5522 - sparse_categorical_accuracy: 0.8966 - val_loss: 0.2439 - val_sparse_categorical_accuracy: 0.9130
Epoch 6/10
116/116 [=======

In [40]:
loss, acc = model.evaluate(X_val, y_val, batch_size = 16)
y_pred = model.predict(X_val)


47/1 [==================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [36]:
print (y_pred)
y_pred = np.argmax(y_pred, axis = 1)
print (y_pred)

[[9.99999642e-01 3.44536517e-07]
 [2.22925053e-04 9.99777138e-01]
 [9.99991417e-01 8.54099108e-06]
 [1.00000000e+00 3.82938588e-08]
 [9.99999642e-01 3.10959592e-07]
 [9.99988675e-01 1.13527658e-05]
 [3.90958985e-06 9.99996066e-01]
 [9.99983788e-01 1.62625511e-05]
 [9.99998808e-01 1.17621039e-06]
 [9.99999285e-01 7.15165527e-07]
 [1.00000000e+00 1.17608501e-08]
 [9.97918785e-01 2.08123284e-03]
 [9.78946802e-04 9.99021053e-01]
 [9.99999762e-01 2.68241422e-07]
 [1.00000000e+00 3.86975429e-08]
 [1.00000000e+00 5.18664685e-08]
 [1.00000000e+00 6.04714279e-10]
 [9.99988198e-01 1.18148018e-05]
 [9.99852180e-01 1.47790255e-04]
 [1.00000000e+00 1.91630245e-09]
 [9.99999762e-01 2.68425168e-07]
 [1.00000000e+00 1.11220855e-08]
 [9.99999762e-01 1.82870338e-07]
 [9.99996662e-01 3.28182773e-06]
 [9.99999642e-01 3.14634093e-07]
 [1.00000000e+00 5.72576262e-08]
 [1.00000000e+00 2.56778554e-09]
 [1.00000000e+00 8.08324074e-10]
 [9.98112321e-01 1.88768702e-03]
 [1.00000000e+00 1.58787756e-08]
 [1.000000

In [37]:
print (y_val)

[0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]


In [ ]:
model, 